In [1]:
# import

import numpy as np
import os
import matplotlib.pyplot as plt
import time
import re
import shutil
from datetime import datetime
from scipy.io import savemat
from scipy.fft import fft
from scipy.interpolate import interp1d
from pathlib import Path

In [2]:
# analysis functions

def get_time(s):
    hr = []; minute = []; sec = [];
    A = [m.start() for m in re.finditer('_', s)];
    year ="".join(i for i in s[A[10]+1:A[11]])
    month ="".join(i for i in s[A[11]+1:A[12]])
    day ="".join(i for i in s[A[12]+1:A[13]])
    hr = "".join(i for i in s[A[14]+1:A[15]])
    minute = "".join(i for i in s[A[15]+1:A[16]])
    sec = s[A[16] + 1]+s[A[16] + 2];
    return year, month, day, hr, minute, sec

def get_current_timestamp(filename):
    y, M, d, h, m, s = get_time(filename)
    if('下' in filename):
        c = 12
    else:
        c = 0
    h = str( int(h)%12 + c )
    current_date = y + '_' + M + '_' + d + ':' + h + ':' + m + ':' + s
    current_stamp = time.mktime(datetime.strptime(current_date,"%Y_%m_%d:%H:%M:%S").timetuple())
    return current_stamp

def check_data_exist(fn_type,nameX,nameY,nameZ,data):
    if(fn_type == 1):
        if not data:
            print('File error: Xfile is empty.')
            print('File error: Yfile is empty.')
            print('File error: Zfile is empty.')
            return 1
        return 0
    if(fn_type == 2):
        if not data:
            print(nameX+' has no time index value.')
            print(nameY+' has no time index value.')
            print(nameZ+' has no time index value.')
            return 1
        return 0
    if(fn_type == 3):
        if(np.max(data)<60000):
            print('Data large missing.')
            return 1
        return 0

def insert_zero(data,index):
    for ind in range(index):
        data = np.insert(data,ind,0)
    return data

def timestamp2timestr(timestamp):
    return datetime.fromtimestamp(timestamp).strftime("%Y-%#m-%#d:%H:%M:%S")

In [3]:
#judge functions

def freq_score(data,ax):
    score = 0
    if(ax == 'X'):
        for i in range(len(data)):
            if(data[i]>=70 and data[i]<=73):
                score = score + 1
            elif(data[i]>=33 and data[i]<=42):
                score = score + 0.67
            else:
                score = score + 0.05
    
    if(ax == 'Y'):
        for i in range(len(data)):
            if(data[i]>=81 and data[i]<=84):
                score = score + 1
            elif(data[i]>=167 and data[i]<=170):
                score = score + 0.3
            else:
                score = score + 0.1
    
    if(ax == 'Z'):
        for i in range(len(data)):
            if(data[i]>=40 and data[i]<=43):
                score = score + 1
            elif((data[i]>=38 and data[i]<=39)or(data[i]==44)):
                score = score + 0.6
            elif(data[i]>=35 and data[i]<=36):
                score = score + 0.25
            else:
                score = score + 0.1
    
    return score

def freq_expression(dataX,dataY,dataZ):
    scoreX = freq_score(dataX,'X')
    scoreY = freq_score(dataY,'Y')
    scoreZ = freq_score(dataZ,'Z')
    X_determine = 0
    Y_determine = 0
    Z_determine = 0
    
    if(scoreX>(3*(len(dataX)/5))):
        X_determine = 1
    if(scoreY>(2.8*(len(dataY)/5))):
        Y_determine = 1
    if(scoreZ>(3.25*(len(dataZ)/5))):
        Z_determine = 1
    
    return (X_determine and Y_determine and Z_determine)

# determine collection situation, report that less than 0.667 or not
def colle_rate(data,total_number):
    if(isinstance(data,list)):
        if(len(data) == 1):
            return ((data/total_number)<0.667)
        elif(len(data) > 1):
            T = [i for i in data if i/total_number<0.667]
            return len(T)
    
    elif(isinstance(data,np.ndarray)):
        if(len(data) == 1):
            return ((data/total_number)<0.667)
        elif(len(data) > 1):
            T = data[(data/total_number)<0.667]
            return len(T)
            
    
#def energy_expression(data):
    
    
#def final_result(data):

In [4]:
# file operations

# Path("/my/directory").mkdir(parents=True, exist_ok=True)

In [5]:
# Some basic settings

Sample_Rate = 400
Sample_Period = 300
r_normal = 0.001
Amp_max = 0.01 # 600/60000
r_normal = 0.001

parent_foldername = 'autotest'
folderpwd = 'D:' + os.sep + '震動資料' + os.sep + parent_foldername
recordpwd = folderpwd + os.sep + 'runrecord.txt'
diarypwd = folderpwd + os.sep + 'diary.txt'
files = [f for f in os.listdir(folderpwd) if os.path.isfile(os.path.join(folderpwd, f))]
if('runrecord.txt' in files):
    files.remove('runrecord.txt')
if('diary.txt' in files):
    files.remove('diary.txt')

f0 = np.linspace(1/Sample_Period, Sample_Rate/2, int(Sample_Period*Sample_Rate/2))

In [6]:
# get last time information from record
if(os.path.exists(recordpwd)):
    last_info = np.loadtxt(recordpwd, dtype = 'int')
    last_file_time = last_info
else:
    record_file = open(recordpwd,'w')
    record_file.close()
    last_file_time = 0

In [7]:
new_files = [i for i in files if ((get_current_timestamp(i)>last_file_time)and(get_current_timestamp(i)<(time.time()-Sample_Period)))]

In [27]:
# Get X Y Z file and run fft

# run setting days
for file in range(1):
    
    # Some settings
    Xfile = []; Yfile = []; Zfile = [];
    FX2 = []; FY2 = []; FZ2 = [];
    XA = []; YA = []; ZA = [];
    NA = []; N_original = []; t_old = [];
    featureX = []; featureY = []; featureZ = [];
    scoreX = []; scoreY = []; scoreZ = [];
    data_total_number = [];
    
    Xfile = [new_files[i] for i in range(len(new_files)) if ('Ch_X' in new_files[i])]
    Yfile = [new_files[i] for i in range(len(new_files)) if ('Ch_Y' in new_files[i])]
    Zfile = [new_files[i] for i in range(len(new_files)) if ('Ch_Z' in new_files[i])]
    
    if check_data_exist(1,_,_,_,min(len(Xfile),len(Yfile),len(Zfile))):
        print('Empty file')
        break
    
    # store data and run for each file
    for file_id in range(len(Xfile)):
        
        X = []; Y = []; Z = [];
        t_x = []; t_y = []; t_z = [];
        
        current_stamp = get_current_timestamp(new_files[file_id])
        print(current_stamp)
        
        # suppose three axis data have same time index and no single axis data loss
        # get three axis' data
        filepwd = folderpwd + os.sep + Xfile[file_id]
        loadx = np.loadtxt(filepwd, skiprows = 22, encoding = 'utf-8', delimiter = ';')
        t_x = [loadx[j][0] for j in range(len(loadx)) if(loadx[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]
        X = [loadx[j][1] for j in range(len(loadx)) if(loadx[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]

        filepwd = folderpwd + os.sep + Yfile[file_id]
        loady = np.loadtxt(filepwd, skiprows = 22, encoding = 'utf-8', delimiter = ';')
        t_y = t_x;#[loady[j][0] for j in range(len(loady)) if(loady[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]
        Y = [loady[j][1] for j in range(len(loady)) if(loady[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]

        filepwd = folderpwd + os.sep + Zfile[file_id]
        loadz = np.loadtxt(filepwd, skiprows = 22, encoding = 'utf-8', delimiter = ';')
        t_z = t_x;#[loadz[j][0] for j in range(len(loadz)) if(loadz[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]
        Z = [loadz[j][1] for j in range(len(loadz)) if(loadz[j][0] < ( (Sample_Rate * Sample_Period) + 10) )]
        
        # move operated file to each folder
        y,M,d,h,m,s = get_time(new_files[file_id])
        class_folder = folderpwd + os.sep + y +'_'+ M +'_'+ d
        Path(class_folder).mkdir(parents=True, exist_ok=True)
        shutil.move(folderpwd + os.sep + Xfile[file_id],class_folder)
        shutil.move(folderpwd + os.sep + Yfile[file_id],class_folder)
        shutil.move(folderpwd + os.sep + Zfile[file_id],class_folder)
        
        
        # check data exist or not
        if check_data_exist(2,Xfile[file_id],Yfile[file_id],Zfile[file_id],t_x):
            continue
            
        if check_data_exist(3,_,_,_,t_x):
            f=open('diary.txt','a')
            f.write(y + '-' + M + '-' + d + '-' + h + ':' + m + ':' + s + 'lots of missing.')
            f.close()
            # print('alarm')
            # break?
        
        N_original.append(len(t_x))
        # interpolate
        t = np.linspace(min(t_x), max(t_x), int(max(t_x) - min(t_x) + 1), dtype = int)
        X = interp1d(t_x,X)(t)
        Y = interp1d(t_y,Y)(t)
        Z = interp1d(t_z,Z)(t)
        
        t_old.append(current_stamp)
        XA.append(X)
        YA.append(Y)
        ZA.append(Z)  
        
        N = len(t)
        NA.append(N)
        TT = N/Sample_Rate
        f = np.linspace(1/TT,Sample_Rate/2,int(N/2))
        L = len(f)
        inds = min([i for (i, val) in enumerate(f0) if val > 1/TT])
        int_freqs = 300*np.linspace(1,200,200,dtype='int')-1
        
        # three axis fft
        F = fft(X)/len(X)
        F = np.delete(F, list(range(L+1,len(F))))
        F = np.delete(F,0)
        FX = interp1d(f,abs(F))(f0[f0 > 1/TT])
        FX = insert_zero(FX,inds)
        FX2.append(FX[int_freqs])
        scoreX.append(freq_score(np.argsort([-x for x in FX[int_freqs]])[:5],'X'))
        
        F = fft(Y)/len(Y)
        F = np.delete(F, list(range(L+1,len(F))))
        F = np.delete(F,0)
        FY = interp1d(f,abs(F))(f0[f0 > 1/TT]);
        FY = insert_zero(FY,inds)
        FY2.append(FY[int_freqs])
        scoreY.append(freq_score(np.argsort([-y for y in FY[int_freqs]])[:5],'Y'))
        
        F = fft(Z)/len(Z)
        F = np.delete(F, list(range(L+1,len(F))))
        F = np.delete(F,0)
        FZ = interp1d(f,abs(F))(f0[f0 > 1/TT])
        FZ = insert_zero(FZ,inds)
        FZ2.append(FZ[int_freqs])
        scoreZ.append(freq_score(np.argsort([-z for z in FZ[int_freqs]])[:5],'Z'))

        
    # check time index existence
    if (len(t_old) == 0):
        print('Error: no time index.')
        break
    
    # save data as mat file
    savemat(class_folder+os.sep+y+'_'+ M +'_'+d+'_'+h+'_'+m+'_'+s+'.mat',mdict={'FX2':FX2,'FY2':FY2,'FZ2':FZ2})
    
    # normal or not, set 0 for no problem
    freq_judgement_value = 0
    if not freq_judgement_value:
        for data_id in range(len(FX2)):
            j_tmp = freq_expression(np.argsort([-x for x in FX2[data_id]])[:5],
                                    np.argsort([-y for y in FY2[data_id]])[:5],
                                    np.argsort([-z for z in FZ2[data_id]])[:5])
            if not j_tmp:
                freq_judgement_value = 1
                break
                
    # collect enough data or not          
    collection_rate = colle_rate(N_original,Sample_Rate*Sample_Period)
            
    # report
    if(freq_judgement_value):
        time_range_min = timestamp2timestr(min(t_old))
        time_range_max = timestamp2timestr(max(t_old))
        print('frequency problem')
        f = open(diarypwd,'a')
        f.write(datetime.fromtimestamp(time.time()).strftime("%Y-%#m-%#d:%H:%M:%S")+'  '+
                str(time_range_min)+' to '+str(time_range_max)+' has frequency problem.\n')
        f.close()
    if(collection_rate):
        time_range_min = timestamp2timestr(min(t_old))
        time_range_max = timestamp2timestr(max(t_old))
        print('data collection problem')
        f = open(diarypwd,'a')
        f.write(datetime.fromtimestamp(time.time()).strftime("%Y-%#m-%#d:%H:%M:%S")+'  '+
                str(time_range_min)+' to '+str(time_range_max)+' has data collection problem.\n')
        f.close() 

1580486412.0
1580486716.0
1580487020.0
frequency problem


In [13]:
# update run record file

last_info_update = max(t_old)
f = open(recordpwd,'w')
f.write(str(last_info_update))
f.close()